In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)   

data = pd.read_csv('DATA_PROVINSI.csv')
data_kecelakaan = data['jumlah_kecelakaan']
X = data_kecelakaan[:-1].values.reshape(-1,1)
y = data_kecelakaan[1:].values.reshape(-1,1).ravel()

df = pd.DataFrame(data)

# Pastikan jumlah sampel dan target sama
assert len(X) == len(y), "Jumlah sampel dan target harus sama"

# Membagi kolom tanggal menjadi tahun dan bulan
df['tahun'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[0]))
df['bulan'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[1]))

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Memisahkan fitur dan target
X = df[['tahun', 'bulan']].values
y = df['jumlah_kecelakaan'].values

In [3]:
# Pisahkan data menjadi training dan testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Fungsi untuk menghitung MAPE menggunakan model SVR dengan parameter tertentu
def calculate_mape(X_train, y_train, X_val, y_val, params):
    C, epsilon, gamma = params
    svr = SVR(kernel='linear', C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred) *100
    return mape

# Inisialisasi populasi awal
def initialize_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        chromosome = []
        for param_range in parameter_ranges:
            param_value = np.random.uniform(param_range[0], param_range[1])
            chromosome.append(param_value)
        population.append(chromosome)
    return population

# Seleksi individu berdasarkan nilai fitness (MAPE)
def selection(population, X_train, y_train, X_val, y_val):
    selected_population = []
    for chromosome in population:
        mape = calculate_mape(X_train, y_train, X_val, y_val, chromosome)
        selected_population.append((chromosome, mape))
    selected_population.sort(key=lambda x: x[1])  # Sort berdasarkan MAPE terendah
    return selected_population[:len(population)//2]  # Pilih setengah individu terbaik

# Crossover: Menghasilkan keturunan dari pasangan individu terpilih
def crossover(selected_population):
    new_population = []
    ukuran_populasi = len(selected_population)
    
    for i in range(ukuran_populasi):
        parent1 = selected_population[i][0]
        
        # Tentukan parent2 dengan pemeriksaan batas
        if i % 2 == 0:
            if i - 1 >= 0:
                parent2 = selected_population[i-1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i-1 tidak valid
        else:
            if i + 1 < ukuran_populasi:
                parent2 = selected_population[i+1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i+1 tidak valid
        
        crossover_point = np.random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)
    
    return new_population

# Mutasi: Mengubah nilai gen secara acak pada populasi baru
def mutation(new_population, mutation_rate, parameter_ranges, sigma_range):
    for i in range(len(new_population)):
        if np.random.random() < mutation_rate:
            mutated_gene_index = np.random.randint(0, len(new_population[i]))
            sigma = np.random.uniform(sigma_range[0], sigma_range[1])
            new_value = new_population[i][mutated_gene_index] + np.random.normal(0, sigma)
            new_population[i][mutated_gene_index] = np.clip(new_value, parameter_ranges[mutated_gene_index][0], parameter_ranges[mutated_gene_index][1])
    return new_population

In [5]:
# Rentang nilai parameter optimal
parameter_ranges = [(0.0001, 0.1), (0.001, 0.1), (1e-07, 0.0001)]

# Inisialisasi populasi awal
population_size = 50
population = initialize_population(population_size, parameter_ranges)

# Iterasi algoritma genetika
crossover_rate = 0.7  # Tetapkan tingkat crossover tetap untuk eksperimen
mutation_rate = 0.3  # Tetapkan tingkat mutasi tetap untuk eksperimen
num_generations = 250  # Tetapkan jumlah generasi tetap untuk eksperimen
sigma_range = (0.001, 0.01)

for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    selected_population = selection(population, X_train, y_train, X_val, y_val)
    for i, (chromosome, mape) in enumerate(selected_population):
        print(f"Individu {i + 1}: C={chromosome[0]}, epsilon={chromosome[1]}, gamma={chromosome[2]}, MAPE={mape}")
    
    new_population = crossover(selected_population)
    new_population = mutation(new_population, mutation_rate, parameter_ranges, sigma_range)

    # Adjust the population size if necessary
    if len(new_population) < population_size:
        additional_individuals = initialize_population(population_size - len(new_population), parameter_ranges)
        new_population.extend(additional_individuals)
    elif len(new_population) > population_size:
        new_population = new_population[:population_size]
    
    # Make sure the new population has the correct size
    assert len(new_population) == population_size, "Ukuran populasi baru tidak sesuai"
    
    population = new_population

# Mengambil individu terbaik (dengan MAPE terendah) dari populasi terakhir
best_chromosome, best_mape = min(selected_population, key=lambda x: x[1])

# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='linear', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nParameter terbaik:")
print(f"C: {best_chromosome[0]}")
print(f"Epsilon: {best_chromosome[1]}")
print(f"Gamma: {best_chromosome[2]}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu 1: C=0.03642659727769147, epsilon=0.09720642618937511, gamma=9.624848476471691e-05, MAPE=27.377962492858725
Individu 2: C=0.08181967511565706, epsilon=0.08621232774237801, gamma=7.945178400659513e-07, MAPE=27.410518604549395
Individu 3: C=0.01873834888271498, epsilon=0.0893633408505078, gamma=5.398028996737351e-05, MAPE=27.48593001393652
Individu 4: C=0.03751655787285152, epsilon=0.0951207163345817, gamma=7.322619478695937e-05, MAPE=27.583418486759747
Individu 5: C=0.0633770352753913, epsilon=0.08727459842858405, gamma=8.038684048222154e-05, MAPE=27.590442598178726
Individu 6: C=0.01620600659667504, epsilon=0.09304006758191474, gamma=8.083122591848525e-05, MAPE=27.603300303969608
Individu 7: C=0.011094187260314909, epsilon=0.02356558109165223, gamma=4.276806808376301e-05, MAPE=27.776760860886252
Individu 8: C=0.025004293691972608, epsilon=0.04162790938052734, gamma=7.557955874045057e-05, MAPE=27.80732081265816
Individu 9: C=0.05112365552749882, epsilon=0.0423236893

In [6]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 7.01915168]
 [ 8.34167139]
 [16.48135008]
 [18.86188554]
 [10.43772175]
 [21.22243985]
 [15.70781941]
 [19.63541621]
 [10.17321782]
 [22.01595167]
 [19.12638949]
 [20.16442408]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='poly', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.33108368 0.33108375 0.33108441 0.33108452 0.33108394 0.33108472
 0.33108428 0.33108465 0.33108392 0.33108476 0.33108453 0.33108467]
MAPE: 58.90881497411423


In [8]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[13.92576157]
 [13.9257636 ]
 [13.92578543]
 [13.9257891 ]
 [13.92576992]
 [13.92579585]
 [13.92578113]
 [13.9257934 ]
 [13.92576951]
 [13.92579708]
 [13.92578951]
 [13.92579422]]


In [9]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='sigmoid', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.33108364 0.33108374 0.33108392 0.3310841  0.33108376 0.33108413
 0.33108401 0.33108402 0.33108374 0.33108419 0.33108412 0.33108406]
MAPE: 58.90883286665146


In [10]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[13.9257602 ]
 [13.92576347]
 [13.92576944]
 [13.92577533]
 [13.92576394]
 [13.92577645]
 [13.92577225]
 [13.92577252]
 [13.92576328]
 [13.92577841]
 [13.92577598]
 [13.92577383]]


In [12]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='rbf', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.3310835  0.33108373 0.33108446 0.33108488 0.33108386 0.33108506
 0.33108456 0.33108478 0.33108382 0.3310852  0.33108493 0.33108487]
MAPE: 58.908782892933665


In [13]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[13.92575536]
 [13.92576312]
 [13.92578717]
 [13.92580113]
 [13.92576752]
 [13.92580707]
 [13.92579053]
 [13.92579777]
 [13.92576596]
 [13.92581173]
 [13.92580268]
 [13.92580087]]
